# 완료 출력을 새로운 시드 매개변수와 일관되게 만드는 방법

**요약**: 이제 개발자는 일관된 완료를 위해 채팅 완료 요청에 `seed` 매개 변수를 지정할 수 있습니다. 개발자가 결정론에 영향을 줄 수 있는 시스템의 변경 사항을 이해하는 데 도움이 되도록 항상 응답에 `system_fingerprint`를 포함합니다.

### 컨텍스트

결정론은 항상 사용자 커뮤니티에서 저희 API를 사용할 때 큰 요청이었습니다. 예를 들어, 결정론적 수치 결과를 얻을 수 있는 기능이 부여되면 사용자는 수치 변화에 민감한 사용 사례의 상당수를 잠금 해제할 수 있습니다.

#### 일관된 출력을 위한 모델 수준 기능

채팅 완료 및 완료 API는 기본적으로 비결정적(즉, 모델 결과가 요청마다 다를 수 있음)이지만 이제 몇 가지 모델 수준 컨트롤을 사용하여 결정적 결과를 일부 제어할 수 있습니다.

이렇게 하면 일관된 완성을 통해 API 위에 구축된 모든 것에 대한 모델 동작을 완벽하게 제어할 수 있으며, 결과를 재현하고 테스트하는 데 매우 유용하므로 정확히 어떤 결과를 얻을 수 있는지 알 수 있어 안심할 수 있습니다.

#### 일관된 출력 구현하기

API 호출 전반에 걸쳐 _거의_ 결정론적 출력을 받으려면:

- seed` 매개변수를 원하는 정수로 설정하되, 모든 요청에서 동일한 값을 사용하세요. 예: `12345`.
- 다른 모든 매개변수(프롬프트, 온도, top_p 등)는 요청 전체에서 동일한 값으로 설정합니다.
- 응답에서 `system_fingerprint` 필드를 확인합니다. 시스템 지문은 OpenAI 서버가 완성을 생성하는 데 사용하는 모델 가중치, 인프라 및 기타 구성 옵션의 현재 조합에 대한 식별자입니다. 요청 매개변수를 변경하거나 OpenAI가 모델을 제공하는 인프라의 수치 구성을 업데이트할 때마다 변경됩니다(1년에 몇 번씩 발생할 수 있음).

시드`, 요청 매개변수, `시스템_지문`이 요청 전반에 걸쳐 모두 일치한다면 모델 출력은 대부분 동일할 것입니다. 컴퓨터의 고유한 비결정성으로 인해 요청 매개변수와 `system_fingerprint`가 일치하더라도 응답이 다를 가능성은 적습니다.


### 일관된 출력을 위한 모델 레벨 컨트롤 - `seed` 및 `system_fingerprint`

##### `seed`

이 값을 지정하면 시스템은 결정론적으로 샘플링하기 위해 최선을 다하므로 동일한 시드와 매개변수로 반복 요청하면 동일한 결과가 반환되어야 합니다. 결정론은 보장되지 않으며, 백엔드의 변경 사항을 모니터링하려면 `system_fingerprint` 응답 파라미터를 참조해야 합니다.

##### `system_fingerprint`

이 핑거프린트는 모델이 실행되는 백엔드 구성을 나타냅니다. 시드 요청 매개변수와 함께 사용하여 결정론에 영향을 줄 수 있는 백엔드 변경이 언제 이루어졌는지 파악할 수 있으며, 이는 사용자가 "거의 항상 동일한 결과"를 기대해야 하는지에 대한 지표가 됩니다.


## 예시: 고정 시드를 사용하여 일관된 단편 스토리 생성하기

이 예에서는 고정 시드를 사용하여 일관된 짧은 스토리를 생성하는 방법을 보여드리겠습니다. 이 방법은 테스트, 디버깅 또는 일관된 출력이 필요한 애플리케이션을 위해 동일한 결과를 재현해야 하는 시나리오에서 특히 유용할 수 있습니다.


In [ ]:
import asyncio
import openai
import pprint
import difflib
from IPython.display import display, HTML

GPT_MODEL = "gpt-3.5-turbo-1106"
openai.api_key = "" # openai api 키 넣으세요

In [ ]:
async def get_chat_response(system_message: str, user_request: str, seed: int = None):
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_request},
        ]

        response = openai.ChatCompletion.create(
            model=GPT_MODEL,
            messages=messages,
            seed=seed,
            max_tokens=200,
            temperature=0.7,
        )

        response_content = response["choices"][0]["message"]["content"]
        system_fingerprint = response["system_fingerprint"] ## gpt-3.5-turbo-1106에서만 먹는것 같습니다.
        prompt_tokens = response["usage"]["prompt_tokens"]
        completion_tokens = (
            response["usage"]["total_tokens"] - response["usage"]["prompt_tokens"]
        )

        table = f"""
        <table>
        <tr><th>Response</th><td>{response_content}</td></tr>
        <tr><th>System Fingerprint</th><td>{system_fingerprint}</td></tr>
        <tr><th>Number of prompt tokens</th><td>{prompt_tokens}</td></tr>
        <tr><th>Number of completion tokens</th><td>{completion_tokens}</td></tr>
        </table>
        """
        display(HTML(table))

        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# 출력 테이블 두개 비교할라고 만든 펑션이라 성능과는 상관없습니다.

def compare_responses(previous_response: str, response: str):
    d = difflib.Differ()
    diff = d.compare(previous_response.splitlines(), response.splitlines())

    diff_table = "<table>"
    diff_exists = False

    for line in diff:
        if line.startswith("- "):
            diff_table += f"<tr style='color: red;'><td>{line}</td></tr>"
            diff_exists = True
        elif line.startswith("+ "):
            diff_table += f"<tr style='color: green;'><td>{line}</td></tr>"
            diff_exists = True
        else:
            diff_table += f"<tr><td>{line}</td></tr>"

    diff_table += "</table>"

    if diff_exists:
        display(HTML(diff_table))
    else:
        print("No differences found.")

먼저 'seed` 매개 변수 없이 '화성으로의 여행'에 대한 짧은 스토리를 생성해 보겠습니다. 이것이 기본 동작입니다:


In [ ]:
#프롬프트 만드는 부분 ( 변경하셔도 됩니다. 토픽이랑 유저리퀘스트 묶어서 들어갑니다.)

topic = "a journey to Mars"
system_message = "You are a helpful assistant that generates short stories."
user_request = f"Generate a short story about {topic}."

previous_response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

# 출력 비교
compare_responses(previous_response, response)

Response,"In the year 2050, a team of courageous astronauts embarked on a groundbreaking mission to Mars. The journey was filled with uncertainty and danger, but the crew was undeterred by the challenges that lay ahead. As their spacecraft hurtled through the vast expanse of space, the astronauts marveled at the beauty of the stars and the distant planets. They passed the time by conducting experiments, training for the mission ahead, and bonding with one another. After months of travel, the red planet finally came into view. The crew prepared for the landing, their hearts pounding with a mix of excitement and nervous anticipation. As the spacecraft touched down on the Martian surface, cheers erupted in the control room back on Earth. The astronauts stepped out onto the alien terrain, taking in the breathtaking landscape of rusty red rocks and dusty plains. They set up their base camp and began their scientific research, collecting samples and conducting experiments to better understand the planet's composition and potential for sustaining life. Despite the challenges of living"
System Fingerprint,fp_fefa7b2153
Number of prompt tokens,31
Number of completion tokens,200


Response,"In the year 2050, a team of astronauts set out on a groundbreaking mission to Mars. The journey was long and arduous, but the crew was determined to make history. As they approached the red planet, they marveled at its otherworldly beauty and the sense of awe and wonder filled their hearts. Upon landing, the astronauts began to explore the alien landscape, conducting scientific experiments and collecting samples. They were amazed by the vast canyons, towering mountains, and the eerie silence that surrounded them. Each step they took was a giant leap for humankind, and they felt a profound sense of accomplishment. As they prepared to return to Earth, the astronauts reflected on the significance of their journey. They knew that their discoveries would pave the way for future generations to explore and inhabit Mars. With their mission complete, they boarded their spacecraft and set their sights on the distant blue planet in the sky, knowing that they had left their mark on the history of space exploration."
System Fingerprint,fp_fefa7b2153
Number of prompt tokens,31
Number of completion tokens,198


"- In the year 2050, a team of courageous astronauts embarked on a groundbreaking mission to Mars. The journey was filled with uncertainty and danger, but the crew was undeterred by the challenges that lay ahead."
"+ In the year 2050, a team of astronauts set out on a groundbreaking mission to Mars. The journey was long and arduous, but the crew was determined to make history. As they approached the red planet, they marveled at its otherworldly beauty and the sense of awe and wonder filled their hearts."
""
"- As their spacecraft hurtled through the vast expanse of space, the astronauts marveled at the beauty of the stars and the distant planets. They passed the time by conducting experiments, training for the mission ahead, and bonding with one another."
"+ Upon landing, the astronauts began to explore the alien landscape, conducting scientific experiments and collecting samples. They were amazed by the vast canyons, towering mountains, and the eerie silence that surrounded them. Each step they took was a giant leap for humankind, and they felt a profound sense of accomplishment."
""
"+ As they prepared to return to Earth, the astronauts reflected on the significance of their journey. They knew that their discoveries would pave the way for future generations to explore and inhabit Mars. With their mission complete, they boarded their spacecraft and set their sights on the distant blue planet in the sky, knowing that they had left their mark on the history of space exploration."
"- After months of travel, the red planet finally came into view. The crew prepared for the landing, their hearts pounding with a mix of excitement and nervous anticipation. As the spacecraft touched down on the Martian surface, cheers erupted in the control room back on Earth."
-
"- The astronauts stepped out onto the alien terrain, taking in the breathtaking landscape of rusty red rocks and dusty plains. They set up their base camp and began their scientific research, collecting samples and conducting experiments to better understand the planet's composition and potential for sustaining life."
-


이제 동일한 주제(화성으로의 여행)의 단편 스토리를 상수 'seed'가 123인 상태로 생성하고 응답과 'system_fingerprint'를 비교해 보겠습니다.


In [ ]:
SEED = 123 ## 시드는 총 9e+18 개 까지 가능 

response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)
previous_response = response
response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)

compare_responses(previous_response, response)

Response,"In the not-so-distant future, a team of brave astronauts embarked on a groundbreaking journey to Mars. The spacecraft, named ""Odyssey,"" soared through the vast expanse of space, leaving Earth behind as they ventured toward the mysterious red planet. As the crew navigated through the cosmos, they encountered a series of challenges and obstacles, from intense solar flares to treacherous asteroid fields. However, their unwavering determination and spirit of camaraderie propelled them forward, overcoming each hurdle with courage and resilience. Upon reaching Mars, the astronauts were greeted by a breathtaking landscape of rust-colored deserts and towering canyons. They marveled at the alien terrain, conducting scientific experiments and collecting samples to better understand the planet's enigmatic history. Amidst their exploration, the crew faced unexpected setbacks, including a sudden dust storm that threatened their safety. Yet, they stood united, devising ingenious solutions and supporting each other through the adversity. After a successful mission on Mars, the"
System Fingerprint,fp_fefa7b2153
Number of prompt tokens,31
Number of completion tokens,200


Response,"In the not-so-distant future, a team of brave astronauts embarked on a groundbreaking journey to Mars. The spacecraft, named ""Odyssey,"" soared through the vast expanse of space, leaving Earth behind as they ventured toward the mysterious red planet. As the crew navigated through the cosmos, they encountered a series of challenges and obstacles, from intense solar flares to treacherous asteroid fields. However, their unwavering determination and spirit of camaraderie propelled them forward, overcoming each hurdle with courage and resilience. Upon reaching Mars, the astronauts were greeted by a breathtaking landscape of rust-colored deserts and towering canyons. They marveled at the alien terrain, conducting scientific experiments and collecting samples to better understand the planet's enigmatic history. Amidst their exploration, the crew faced unexpected setbacks, including a sudden dust storm that threatened their safety. Yet, they stood united, devising ingenious solutions and supporting each other through the adversity. After a successful mission on Mars, the"
System Fingerprint,fp_fefa7b2153
Number of prompt tokens,31
Number of completion tokens,200


No differences found.


결론 ## 결론

고정 정수 '시드'를 사용하여 모델에서 일관된 출력을 생성하는 방법을 보여드렸는데, 이는 재현성이 중요한 시나리오에서 특히 유용합니다. 그러나 '시드'가 일관성을 보장하지만 출력의 품질을 보장하지는 않는다는 점에 유의해야 합니다. 예를 들어, 제공된 예제에서는 동일한 시드를 사용하여 화성으로의 여행에 대한 짧은 스토리를 생성했습니다. 모델을 여러 번 쿼리했음에도 불구하고 출력은 일관성을 유지하여 재현성을 위해 이 모델 수준 제어를 사용하는 것이 효과적이라는 것을 보여주었습니다. 이를 확장하여 다른 프롬프트나 모델의 성능을 벤치마킹/평가할 때 일관된 '시드'를 사용하면 각 버전이 동일한 조건에서 평가되어 비교가 공정하고 결과를 신뢰할 수 있습니다.

추가 결론
한국어는 영어처럼 잘 안됩니다. 200자 내외에서도 아직은 만족할만한 시드 고정은 어려웠습니다. 처음 두~세문장 정도는 동일한 출력을 기대해 볼 수 있습니다.
매개변수(프롬프트, 온도, top_p 등)는 가능하면 기본값 (0.7, 200)을 권장합니다. 토큰이 길어지거나 자유도를 변경했을때 영어 출력도 흔들리는 것을 볼 수 있었습니다.
